## Voice response Chatbot using TF-IDF

In [1]:
# Importing Libraries

import random
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
import pyttsx3 # For voice assistance

In [5]:
# Creating chatbot using TF-IDF

# Reading input file
f=open('chatbot.txt','r',errors = 'ignore')
data = f.read()
sw = stopwords.words('english')

# Tokenization - converts to list of sentences
sent_tokens = nltk.sent_tokenize(data) 

def greeting(word): 
    # Define Greetings
    GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "how are you", "how are you doing")
    GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me", 
                          "I'm Good, Thankyou for asking"]
    if word.lower() in GREETING_INPUTS[6:8]:
        return (GREETING_RESPONSES[-1])
    if word.lower() in GREETING_INPUTS[0:6]:
        return random.choice(GREETING_RESPONSES[:-1])

# Generating response
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    len_ = len(sent_tokens)
    #print(len_)
    corpus = []
    
    for i in range(0,len_):
        review = re.sub('[^a-zA-Z0-9]',' ',sent_tokens[i]) # keep alphebets and numbers only
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(sw)]
        review = ' '.join(review)
        corpus.append(review)

    tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=500)
    tfidf = tfidf.fit_transform(corpus).toarray()
    #print(tfidf.shape)
    
    X = (tfidf[-1]).reshape(-1,1)
    vals = cosine_similarity(X.T, tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

def main():
    flag=True
    answer = ''
    print ("Bot: My name is Gems Bot. I will answer your queries about Chatbots. If you want to exit, type Bye!")
    speak("Hello! My name is Gems Bot.I will answer your queries about Chatbots. If you want to exit, type Bye!")
    while(flag==True):
        user_response = input()
        user_response=user_response.lower()
        if(user_response!='bye'):
            if(user_response=='thanks' or user_response=='thank you' ):
                flag=False
                print("Bot: You are welcome..")
                speak("You are welcome")
            else:
                if(greeting(user_response)!=None):
                    answer = greeting(user_response)
                    print("Bot: "+answer)
                    speak(answer)
                else:
                    answer = response(user_response)
                    print("Bot: ",end="")
                    print(answer)
                    speak(answer)
                    sent_tokens.remove(user_response)
        else:
            flag=False
            print("Bot: Bye! take care..")  
            speak("It was nice talking to you! Stay safe")

def speak(answer1):
    spk = pyttsx3.init()

    rate = spk.getProperty('rate')   # getting details of current speaking rate
    spk.setProperty('rate', 150)     # setting up new voice rate

    """VOICE"""
    voices = spk.getProperty('voices')       #getting details of current voice
    spk.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
    spk.say(answer1)
    spk.runAndWait()
    spk.stop()
    
main()

from IPython.display import clear_output
clear_output()